## Install the required libraries

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [47]:
alerts = pd.read_csv("classified_alerts.csv")

In [48]:
cls = set(alerts['class'])

In [49]:
print(cls)

{'a.c.t.b.r.s.e.customexceptionhandler', 'o.s.w.c.httpservererrorexception.internalservererror', 'o.s.w.c.httpservererrorexception.gatewaytimeout', 'a.c.t.b.r.c.l.loggingaspect', 'a.c.t.b.r.s.m.fromtarget.targetresponsedata', 'a.c.t.b.r.s.e.customexceptionhandler.handlehttpstatuscodeexception', 'j.lang.nullpointerexception', 'o.json.jsonexception', 'o.s.w.c.httpservererrorexception.badgateway', 'c.f.j.d.e.unrecognizedpropertyexception.from', 'a.c.t.b.r.c.u.commonutils'}


In [50]:
class_lst = list(cls)
class_lst.index(class_lst[3])

3

In [51]:
len(alerts)

10912

In [52]:
alerts.columns

Index(['appname', 'alertid', 'class', 'words'], dtype='object')

In [53]:
# Convert class to number
def category2Num(cat):
    global class_lst
    return class_lst.index(cat)
    
alerts['category_num'] = alerts['class'].apply(category2Num)

In [54]:
alerts.head()

,appname,alertid,class,words,category_num
0,spg-service-request-inbound-consumer,a1642502490828,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler requestid ...,0
1,spg-service-request-inbound-consumer,a1642502490829,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler customexce...,0
2,spg-service-request-inbound-consumer,a1642502490838,o.s.w.c.httpservererrorexception.badgateway,oswchttpservererrorexceptionbadgateway 502 bad...,8
3,spg-service-request-inbound-consumer,a1642502490888,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler customexce...,0
4,spg-service-request-inbound-consumer,a1642502490889,a.c.t.b.r.s.e.customexceptionhandler,info actbrsecustomexceptionhandler requestid ...,0


In [55]:
!pip install textblob
!pip install wordcloud

In [56]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt

In [57]:
from string import punctuation
from sklearn import svm
from textblob import Word
from wordcloud import WordCloud
import textblob
from textblob import TextBlob

In [58]:
from imblearn.over_sampling import SMOTE

In [59]:
alerts = alerts.drop(['appname', 'alertid', 'class'], axis=1)
alerts.columns

Index(['words', 'category_num'], dtype='object')

In [60]:
alerts['category_num'].value_counts()

4     4262
5     3595
9     1287
0      544
1      340
10     272
8      204
6      136
2      136
7       68
3       68
Name: category_num, dtype: int64

In [61]:
X = alerts['words']
y = alerts['category_num']

In [62]:
print(X)

0        info actbrsecustomexceptionhandler  requestid ...
1        info actbrsecustomexceptionhandler  customexce...
2        oswchttpservererrorexceptionbadgateway 502 bad...
3        info actbrsecustomexceptionhandler  customexce...
4        info actbrsecustomexceptionhandler  requestid ...
                               ...                        
10907    cfjdexcunrecognizedpropertyexception unrecogni...
10908    warn oswsmmaexceptionhandlerexceptionresolver ...
10909    cfjdexcunrecognizedpropertyexception unrecogni...
10910    cfjdexcunrecognizedpropertyexception unrecogni...
10911    warn oswsmmaexceptionhandlerexceptionresolver ...
Name: words, Length: 10912, dtype: object


my_stop_words = set(['info', 'of', 'and', 'cannot', 'warn', 'is', 'not', 'out', 'doctype', 'html', 'failed', 'failure', \
                     'handle', 'request', 'response', 'throwing', 'line', 'get', 'exception', 'detail', 'details', \
                     'be', 'to', 'in', 'public', 'throws', 'known', 'at', 'from', 'with', 'class', 'marked', 'as', 
                     'it', 'this', 'return', 'returning', 'notfound', 'type', 'title'])

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= my_stop_words)

X = tfidf.fit_transform(["this is an apple oswclientHttpServerErrorExceptionBadGateway that was cider", \
                              "this is and actbrseCustomExceptionHandler",\
                              "cfjdeUnrecognizedPropertyException Unrecognized field error class actbrsmfromtargetTargetResponseData not marked as ignorable 4 known properties status code message incidentId"\
                             ])
idf_values = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

#### printing the tfidf vectors
print(X)

#### printing the vocabulary
print(tfidf.vocabulary_)

In [63]:
alerts.head()

,words,category_num
0,info actbrsecustomexceptionhandler requestid ...,0
1,info actbrsecustomexceptionhandler customexce...,0
2,oswchttpservererrorexceptionbadgateway 502 bad...,8
3,info actbrsecustomexceptionhandler customexce...,0
4,info actbrsecustomexceptionhandler requestid ...,0


In [64]:
my_stop_words = set(['info', 'of', 'and', 'cannot', 'warn', 'is', 'not', 'out', 'doctype', 'html', \
                     'failed', 'failure', 'handle', 'request', 'response', 'throwing', 'line', 'get', \
                     'exception', 'detail', 'details', 'be', 'to', 'in', 'public', 'throws', 'known', \
                     'at', 'from', 'with', 'class', 'marked', 'as', 'it', 'this', 'return', 'returning', \
                     'notfound', 'type', 'title'])

In [65]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= my_stop_words)

In [66]:
#Handle class imbalance
X_features = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=67)
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)

In [77]:
def class_balanced_model_fit(X_train_smote, y_train_smote, X_test_smote, y_test_smote, ml_model, coef_show):
    classification = ml_model.fit(X_train_smote, y_train_smote)
    classification_pred = classification.predict(X_test_smote)
    accuracy = classification.score(X_test_smote, y_test_smote)
    model_performance = classification_report(y_test_smote, classification_pred)
    validation_pred_proba_grad = classification.predict_proba(X_test_smote)
    #roc_auc = roc_auc_score(y_test_smote, validation_pred_proba_grad[:,1])
    
    #print("***** Accuracy of the classification model: ", accuracy, " *******")
    #print('')
    #print(model_performance)
    #print('')
    #print("***** ROC_AUC score: ", roc_auc, " *******")
    #print("*************************************************")
    
    if coef_show == 1:
        featureNames = tfidf.get_feature_names()
        coef = classification.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : featureNames, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print("************ Top 10 positive features (variables) ************")
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print("************ Top 10 negative features (variables) ************")        
        print(coeff_df.tail(20).to_string(index=False))
    
    return classification

In [78]:
LRClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500),
                                     coef_show=1)


************ Top 10 positive features (variables) ************
                                Word  Coefficient
                                 400     2.496726
       actbrsecustomexceptionhandler     2.496726
                          badrequest     2.496726
                         deserialize     2.496726
                       errorresource     2.496726
                            instance     2.496726
                          startarray     2.496726
                               token     2.496726
                         jlangstring     1.796090
                           requestid     1.639439
                                 404     1.289124
              customexceptionhandler     1.289124
                               found     1.289124
               handlecustomexception     1.289124
                             message     0.496281
                              status     0.496281
                            endpoint    -0.103568
                          registered

In [79]:
X_test_smote

<9559x62 sparse matrix of type '<class 'numpy.float64'>'
	with 61115 stored elements in Compressed Sparse Row format>

In [82]:
#Test some random text for classification - Do reviews classification as good or bad sentiment. 
Xfeatu = tfidf.transform(
['cfjdexcunrecognizedpropertyexception unrecognized field error class actbrsmfromtargettargetresponsedata not marked as ignorable 4 known properties status code message incidentid', 
'warn oswsmmaexceptionhandlerexceptionresolver  failure in exceptionhandler public oshttpresponseentity jlangobject actbrsecustomexceptionhandlerhandlehttpstatuscodeexception oswclienthttpstatuscodeexception throws jioioexception',
'info actbrsecustomexceptionhandler  customexceptionhandler handlecustomexception is returning 404 notfound not found errorresource type title badrequest status 400 message cannot deserialize instance of jlangstring out of startarray token'])


In [83]:
print(LRClassModel.predict(Xfeatu))

[4 5 0]


### END OF TELSTRA CODE

In [2]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
my_stop_words = set(['apple', 'and', 'this', 'is', 'an', 'that', 'was', 'as', 'known', 'class', 'not', \
                     'marked', 'properties', 'ignorable', 'marked'])
vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)
X = vectorizer.fit_transform(["this is an apple oswclientHttpServerErrorExceptionBadGateway that was cider", \
                              "this is and actbrseCustomExceptionHandler",\
                              "cfjdeUnrecognizedPropertyException Unrecognized field error class actbrsmfromtargetTargetResponseData not marked as ignorable 4 known properties status code message incidentId"\
                             ])
idf_values = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

# printing the tfidf vectors
print(X)

# printing the vocabulary
print(vectorizer.vocabulary_)

  (0, 3)	0.7071067811865476
  (0, 9)	0.7071067811865476
  (1, 0)	1.0
  (2, 7)	0.33333333333333337
  (2, 8)	0.33333333333333337
  (2, 4)	0.33333333333333337
  (2, 10)	0.33333333333333337
  (2, 1)	0.33333333333333337
  (2, 5)	0.33333333333333337
  (2, 6)	0.33333333333333337
  (2, 11)	0.33333333333333337
  (2, 2)	0.33333333333333337
{'oswclienthttpservererrorexceptionbadgateway': 9, 'cider': 3, 'actbrsecustomexceptionhandler': 0, 'cfjdeunrecognizedpropertyexception': 2, 'unrecognized': 11, 'field': 6, 'error': 5, 'actbrsmfromtargettargetresponsedata': 1, 'status': 10, 'code': 4, 'message': 8, 'incidentid': 7}


In [1]:
# test classification dataset
from collections import Counter
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# summarize the dataset
print(X.shape, y.shape)
print(Counter(y))

(1000, 10) (1000,)
Counter({1: 334, 2: 334, 0: 332})


In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [5]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [58]:
from sklearn.feature_extraction import text
text.ENGLISH_STOP_WORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [49]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.681 (0.042)


In [50]:
# make a prediction with a multinomial logistic regression model
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# fit the model on the whole dataset
model.fit(X, y)
# define a single row of input data
row = [1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426]
# predict the class label
yhat = model.predict([row])
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

Predicted Class: 1


In [52]:
row = [2.89149379, -1.39847585, 0.63856893, 1.01647165, 3.51892395, -1.52651223, 2.80998823, -0.58810926, 0.22542177, 0.62835426]
# predict the class label
yhat = model.predict([row])
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

Predicted Class: 2


In [53]:
row = [-3.89149379, 1.39847585, -1.63856893, 3.01647165, 2.51892395, -4.52651223, -3.80998823, 0.68810926, -3.02542177, -3.52835426]
# predict the class label
yhat = model.predict([row])
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

Predicted Class: 1


In [57]:
row = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
# predict the class label
yhat = model.predict([row])
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

Predicted Class: 0


In [51]:
# predict probabilities with a multinomial logistic regression model
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# fit the model on the whole dataset
model.fit(X, y)
# define a single row of input data
row = [1.89149379, -0.39847585, 1.63856893, 0.01647165, 1.51892395, -3.52651223, 1.80998823, 0.58810926, -0.02542177, -0.52835426]
# predict a multinomial probability distribution
yhat = model.predict_proba([row])
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % yhat[0])

Predicted Probabilities: [0.16470456 0.50297138 0.33232406]


### END OF CODE

In [8]:
!pip install textblob
!pip install wordcloud

In [9]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [10]:
import re

In [11]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt

In [12]:
from string import punctuation
from sklearn import svm
from textblob import Word
from wordcloud import WordCloud
import textblob
from textblob import TextBlob

In [13]:
from imblearn.over_sampling import SMOTE

## Task 1: Data Cleaning and Pre-Processing

### Lets do some EDA. Load the data. Check for the missing values. Treat them. Impute if necessary or drop the fields. Get a sense of data imbalance too.

In [14]:
reviews_df = pd.read_csv("sample30.csv")

In [15]:
print(type(reviews_df))

<class 'pandas.core.frame.DataFrame'>


In [16]:
print(reviews_df.columns)
print(reviews_df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


### We need only the 'name', 'reviews_rating', 'reviews_text', 'reviews_username' and 'user_sentiment'. Rest of the columns can be dropped.

In [17]:
reviews_df = reviews_df.drop(['id', 'brand', 'categories', 'manufacturer', 
                              'reviews_date', 'reviews_didPurchase', 'reviews_doRecommend', 
                              'reviews_title', 'reviews_userCity', 'reviews_userProvince'], axis=1)

In [18]:
reviews_df

,name,reviews_rating,reviews_text,reviews_username,user_sentiment
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,i love this album. it's very good. more to the...,joshua,Positive
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor. This review was collected as part...,dorothy w,Positive
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor.,dorothy w,Positive
3,K-Y Love Sensuality Pleasure Gel,1,I read through the reviews on here before look...,rebecca,Negative
4,K-Y Love Sensuality Pleasure Gel,1,My husband bought this gel for us. The gel cau...,walker557,Negative
...,...,...,...,...,...
29995,L'or233al Paris Elvive Extraordinary Clay Reba...,5,I got this conditioner with Influenster to try...,laurasnchz,Positive
29996,L'or233al Paris Elvive Extraordinary Clay Reba...,5,"I love it , I received this for review purpose...",scarlepadilla,Positive
29997,L'or233al Paris Elvive Extraordinary Clay Reba...,5,First of all I love the smell of this product....,liviasuexo,Positive
29998,L'or233al Paris Elvive Extraordinary Clay Reba...,5,I received this through Influenster and will n...,ktreed95,Positive


#### There are ~25K users in in the dataset.

In [19]:
reviews_df['reviews_username'].value_counts()

byamazon customer    41
mike                 41
chris                32
lisa                 16
rick                 15
                     ..
ssttaarr              1
arty                  1
touthao               1
crysteelwiech         1
newmommy25            1
Name: reviews_username, Length: 24914, dtype: int64

#### There are 271 unique products in the dataset.

In [20]:
reviews_df['name'].value_counts()

Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total                         8545
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd                   3325
Clorox Disinfecting Bathroom Cleaner                                              2039
L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz    1186
Planes: Fire Rescue (2 Discs) (includes Digital Copy) (blu-Ray/dvd)               1143
                                                                                  ... 
Citrus Magic Instant Spot & Stain Remover                                            1
Pacific Natural Foods Organic Beef Broth                                             1
Black Sister's Revenge (dvd)                                                         1
Roommates No Place Like Home Peel Stick Wall Decals                                  1
Sloan Royal Urinal Flush Valve, 1.0 Gpf, Royal 186-1                                 1
Name: name, Length: 271, dtype: int64

In [21]:
reviews_df['reviews_username'] = reviews_df['reviews_username'].str.lower()

In [22]:
reviews_df.head()

,name,reviews_rating,reviews_text,reviews_username,user_sentiment
0,Pink Friday: Roman Reloaded Re-Up (w/dvd),5,i love this album. it's very good. more to the...,joshua,Positive
1,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor. This review was collected as part...,dorothy w,Positive
2,Lundberg Organic Cinnamon Toast Rice Cakes,5,Good flavor.,dorothy w,Positive
3,K-Y Love Sensuality Pleasure Gel,1,I read through the reviews on here before look...,rebecca,Negative
4,K-Y Love Sensuality Pleasure Gel,1,My husband bought this gel for us. The gel cau...,walker557,Negative


#### There are no empty review rows.

In [23]:
print(reviews_df['reviews_text'].isnull().sum())

0


#### There are no empty review ratings.

In [24]:
print(reviews_df['reviews_rating'].isnull().sum())

0


### There are no records to be imputed or dropped because all of them seem to have values. We don't have to drop records for products. At the same time, the below list shows that there are around ~50 products that have just 1 reviews. These can be dropped as they qualify to be outliers, but we choose to retain them.

In [25]:
l = reviews_df['name'].value_counts().sort_values()[0:60]
l

Sloan Royal Urinal Flush Valve, 1.0 Gpf, Royal 186-1                                                    1
Disney174 Jake And The Neverland Pirates 4 Piece Bedding Set - Toddler                                  1
Every Man Jack Pomade Signature Mint Scent                                                              1
Clorox Ultimate Care Premium Bleach                                                                     1
Pearhead Id Bracelet Frame                                                                              1
Noosa Honey Yogurt                                                                                      1
Wedding Wishes Wedding Guest Book                                                                       1
Iman Luxury Moisturizing Lipstick, Black Brandy 006                                                     1
La Tortilla Factory Hand Made Style Tortillas Flour                                                     1
Pink Friday: Roman Reloaded Re-Up (w/dvd)     

In [26]:
reviews_df['reviews_text']

0        i love this album. it's very good. more to the...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_text, Length: 30000, dtype: object

## Task 2: Text Processing

### Check if the reviews have any hypertext references.

In [27]:
def checkHyperText(x):
    if x.find("http:") == -1:
        return 0
    else:
        return 1
    
http = reviews_df['reviews_text'].apply(checkHyperText)
http.value_counts()

0    29994
1        6
Name: reviews_text, dtype: int64

### There are a few, hyperlinks, but they are descriptive. Remove the hyperlinks in the reviews text.

In [28]:
reviews_df['reviews_tokenized'] = reviews_df['reviews_text'].apply(lambda x: re.sub(r"http\S+", "", x))
reviews_df['reviews_tokenized']

0        i love this album. it's very good. more to the...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### I applied Lemmatization and found that it had a negative impact on the review classification and the accuracy came down. I choose to convert contractions programmatically - explicitly.

In [29]:
def expand_contractions(s):
    s = re.sub(r"won't", "will not", s)
    s = re.sub(r"would't", "would not", s)
    s = re.sub(r"could't", "could not", s)
    s = re.sub(r"\'d", " would", s)
    s = re.sub(r"can\'t", "can not", s)
    s = re.sub(r"n\'t", " not", s)
    s= re.sub(r"\'re", " are", s)
    s = re.sub(r"\'s", " is", s)
    s = re.sub(r"\'ll", " will", s)
    s = re.sub(r"\'t", " not", s)
    s = re.sub(r"\'ve", " have", s)
    s = re.sub(r"\'m", " am", s)
    return s

reviews_df['reviews_tokenized'] =reviews_df['reviews_tokenized'].apply(lambda x:expand_contractions(x))
reviews_df['reviews_tokenized']

0        i love this album. it is very good. more to th...
1        Good flavor. This review was collected as part...
2                                             Good flavor.
3        I read through the reviews on here before look...
4        My husband bought this gel for us. The gel cau...
                               ...                        
29995    I got this conditioner with Influenster to try...
29996    I love it , I received this for review purpose...
29997    First of all I love the smell of this product....
29998    I received this through Influenster and will n...
29999    I received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### Tokenize the reviews text and retain only the English alphabet - words.

In [30]:
#Tokenize the review text to words
reg = re.compile('[^a-zA-Z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

reviews_df['reviews_tokenized'] = reviews_df['reviews_tokenized'].apply(tokenizeReview)
reviews_df['reviews_tokenized']

0        i love this album it is very good more to the ...
1        good flavor this review was collected as part ...
2                                              good flavor
3        i read through the reviews on here before look...
4        my husband bought this gel for us the gel caus...
                               ...                        
29995    i got this conditioner with influenster to try...
29996    i love it i received this for review purposes ...
29997    first of all i love the smell of this product ...
29998    i received this through influenster and will n...
29999    i received this product complimentary from inf...
Name: reviews_tokenized, Length: 30000, dtype: object

### Compare the original reviews text and pre-processed reviews text that will be fed to the next stage.

In [31]:
reviews_df[['reviews_text', 'reviews_tokenized']]

,reviews_text,reviews_tokenized
0,i love this album. it's very good. more to the...,i love this album it is very good more to the ...
1,Good flavor. This review was collected as part...,good flavor this review was collected as part ...
2,Good flavor.,good flavor
3,I read through the reviews on here before look...,i read through the reviews on here before look...
4,My husband bought this gel for us. The gel cau...,my husband bought this gel for us the gel caus...
...,...,...
29995,I got this conditioner with Influenster to try...,i got this conditioner with influenster to try...
29996,"I love it , I received this for review purpose...",i love it i received this for review purposes ...
29997,First of all I love the smell of this product....,first of all i love the smell of this product ...
29998,I received this through Influenster and will n...,i received this through influenster and will n...


In [32]:
reviews_df.shape

(30000, 6)

In [33]:
reviews_df = reviews_df.drop(['reviews_text'], axis=1)

In [34]:
reviews_df.shape

(30000, 5)

In [35]:
reviews_df['reviews_tokenized'].head(5)

0    i love this album it is very good more to the ...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

### user_sentiment column as 89% positive reviews and just 11% negative reviews. This is huge imbalance in data that would affect the learning process.

In [36]:
reviews_df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [37]:
# Convert 'positive' to 1 and 'negative' to 0
def sentiment2Number(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = reviews_df['reviews_tokenized']
y = reviews_df['user_sentiment'].apply(sentiment2Number)

In [38]:
print(len(X))
print(len(y))

30000
30000


In [39]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

## Task 3: Feature Extraction

### Use TF-IDF Vectorizer and leverage the built-in stop_words functionality and remove stop words too. Use the max_features as a hyper-parameter.

In [40]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

#### As indicated earlier, we have a highly skewed data-set, which is not in balance.

In [41]:
#Indicates class imbalance
y.value_counts()

1    26632
0     3368
Name: user_sentiment, dtype: int64

### Train and test data split.
### Use SMOTE to fix class imbalance before training the model.

In [42]:
#Handle class imbalance
X_features = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=67)
sm = SMOTE()
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)
X_test_smote, y_test_smote = sm.fit_resample(X_test, y_test)

### Core function of the project. Fit the tf-idf vectorized features into the text classification model. Evaluate it with the ROC - AUC method.

In [43]:
def class_balanced_model_fit(X_train_smote, y_train_smote, X_test_smote, y_test_smote, ml_model, coef_show):
    classification = ml_model.fit(X_train_smote, y_train_smote)
    classification_pred = classification.predict(X_test_smote)
    accuracy = classification.score(X_test_smote, y_test_smote)
    model_performance = classification_report(y_test_smote, classification_pred)
    validation_pred_proba_grad = classification.predict_proba(X_test_smote)
    roc_auc = roc_auc_score(y_test_smote, validation_pred_proba_grad[:,1])
    
    print("***** Accuracy of the classification model: ", accuracy, " *******")
    print('')
    print(model_performance)
    print('')
    print("***** ROC_AUC score: ", roc_auc, " *******")
    print("*************************************************")
    
    if coef_show == 1:
        featureNames = tfidf.get_feature_names()
        coef = classification.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : featureNames, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print("************ Top 10 positive features (variables) ************")
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print("************ Top 10 negative features (variables) ************")        
        print(coeff_df.tail(20).to_string(index=False))
    
    return classification

## Task 4: Model Building, with 3 models - Logistic Regression, Bayes and Random Forest.

### Use Logistic Regression as the classification model and evaluate.
#### Accuracy is 94% and AUC is 0.98

In [44]:
LRClassModel = class_balanced_model_fit(X_train_smote, y_train_smote, 
                                     X_test_smote, y_test_smote, 
                                     LogisticRegression(solver='lbfgs', max_iter=500),
                                     coef_show=1)

***** Accuracy of the classification model:  0.9416977611940298  *******

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      5360
           1       0.96      0.92      0.94      5360

    accuracy                           0.94     10720
   macro avg       0.94      0.94      0.94     10720
weighted avg       0.94      0.94      0.94     10720


***** ROC_AUC score:  0.9822858480452218  *******
*************************************************

************ Top 10 positive features (variables) ************
      Word  Coefficient
     great    17.780394
      love    15.116278
      best    12.266600
      good    11.035214
      easy    10.771146
     clean    10.731221
    better     9.063239
     loved     8.598145
      nice     8.314935
 excellent     8.125357
   perfect     7.879954
   awesome     7.312009
   amazing     7.163275
   enjoyed     6.968860
  favorite     6.916870
 wonderful     6.270315
      free     6.074108
  

### Create a few sentences using the above positive and negative words and evaluate if they are classified correctly with Logistic Regression.

In [45]:
#Test some random text for classification - Do reviews classification as good or bad sentiment. 
Xfeatu = tfidf.transform(['i love this album it is very good', 
                          'an easy to use product',
                          'an awful product',
                          'they gave a free gift with the product',
                          'i went crazy after applying the lotion'])
print(LRClassModel.predict(Xfeatu))

[1 1 0 1 0]


### Task 5 - Part A - User-based collaborative recommendation system

In [46]:
#Do reviews classification as good or bad sentiment. 
tfidfFeatures = tfidf.transform(reviews_df.reviews_tokenized)
sentcls = LRClassModel.predict(tfidfFeatures)

#### END OF FILE